# INF8111 - Data Mining

## TP1 Fall 2021 - Duplicate Bug Report Detection

##### Due date: 26/09 at 23:55
##### The TP mark will be penalized by 10 points if  the notebook  takes more than 1 hour to be executed. 

##### Team Members:

    - Name (student id) 1
    - Name (student id) 2
    - Name (student id) 3
    
    
##### Deliverables:

You must submit two separate files to Moodle:
1. This notebook
2. A [Json](https://en.wikipedia.org/wiki/JSON) containing scraped web-page content (bug_reports.json)




## 1 - Overview


Due to complexities of software systems, software bugs are prevalent. Companies, especially large ones, usually use a Bug Tracking System (BTS), also called Issue Tracking System, to manage and track records of bugs. Besides developers and testers, many projects, mainly open source projects, allow users to report new bugs in their BTS.
To do that, users have to fill a form with multiple fields. An important subset of
these fields provides categorical data and only accepts values that range from a fixed list of
options (e.g., component, version and product of the system). Another two important fields
are the summary and the description. The users are free to write anything in both fields
and the only constraint is that the summary has a maximum number of characters. The
submission of a form creates a page, called bug report or issue report, which contains all
the information about a bug.

Due to the lack of communication and syncronization, users may not be aware that a specific bug was already submitted and report it again. Identifying duplicate bug reports is important task in the BTSs and it is the subject of this TP. Basically, our goal is to develop a system that will predicte wheather a pair of new bug report and submitted one is duplicate or not. This system will be used to manually identify duplicate reports.

# 2 Setup

Please run the code below to install the packages needed for this assignment.

In [ ]:
                                                                                                                                                                                                                # If you want, you can use anaconda and install after nltk library
# pip install --user numpy
# pip install --user scikit-learn
# pip install --user nltk
# pip install --user tqdm


#python
#import nltk
#nltk.download("punkt")

# 3 - Data

Please download the zip file in the following url: https://www.dropbox.com/s/s53fqz29z8ch4ip/data.zip?dl=0

In this zip file, there are: 

1. training.txt: This file contains pairs of bug reports that will be used to train our system.
2. validation.txt: This file contains  pairs of bug reports that will be used to evaluate our system.
2. bug_reports: It is a folder that contains the bug report html source. Each html file name follows the pattern **bug_report_id.html**.


Figure below depicts an bug report page example:

In [ ]:
from IPython import display
display.Image("https://irving-muller.github.io/images/bug-report-eclipse.png")

- A : bug report id
- B : creation date
- C : summary
- D : product
- E : component
- F : the report id which the bug report is duplicate
- G : description

The following script loads the test dataset and define some global variables:

In [ ]:
import os

# define the folder path that contain the data
# FOLDER_PATH = "Define folder path that contain threads folder,training_eclipse.txt, and validation.txt"
FOLDER_PATH = "data/"
PAGE_FOLDER = os.path.join(FOLDER_PATH, 'bug_reports')


# Load the evaluation dataset
import json


training_file = open(os.path.join(FOLDER_PATH, "training.txt"))
validation_file = open(os.path.join(FOLDER_PATH, "validation.txt"))
word_vec_path = os.path.join(FOLDER_PATH, "glove.42B.300d_clear.txt")

def read_dataset(f):
    for line in f:
        line = line.strip()
        
        if len(line) == 0:
            continue
        
        rep1, rep2, label = line.split(',')

        rep1 = int(rep1)
        rep2 = int(rep2)
        label = 1.0 if int(label) > 0 else 0.0 
        
        yield (rep1, rep2, label)
    
    


training_pairs = list(read_dataset(training_file))
validation_pairs = list(read_dataset(validation_file))

training_reports_set = set()


for r1, r2, _ in training_pairs:
    training_reports_set.add(r1)
    training_reports_set.add(r2)

# 4 - Web scraping

Web scraping consists in extracting relevant data from pages and prepare it for computational analysis.


## 4.1 - Question 1 (4 points)

Implement extract_data_from_page function. This function extracts the following information from the html: bug report id, creation date, title, product, component, the report id which it is duplicate and description.

*extract_data_from_page* function returns a dictionary with the following structure:


 ```python
{"report_id": int, 
  "dup_id": int or None (the report id which it is duplicate), 
  "component": string, 
  "product": string, 
  "summary": string, 
  "description": string, 
  "creation_date": int}
 ```
For instance, when extract_data_from_page receives "bug_report/18431.html", it returns:
 
```python
{'report_id': 18431,
 'dup_id': 27227,
 'component': 'GEF-Legacy Draw2d',
 'product': 'GEF',
 'summary': 'Polylines ignored by FLowLayout',
 'description': 'I tried a poor sample but it\'s not working the way I thought it would be. Look \nat the following source. I have a figure with FlowLayout and add 2 polylines to \nit but the polylines aren\'t arranged in FlowLayout. Both polylines are pushed \ninto the first flow element. Any ideas why?\n\n\n\npublic static void main(String args[])\n{\n Shell shell = new Shell();\n shell.open();\n shell.setText("draw2d Figures");\n\n LightweightSystem lws = new LightweightSystem(shell);\n\n IFigure panel = new Figure();\n panel.setLayoutManager( new FlowLayout(FlowLayout.HORIZONTAL) );\n\n lws.setContents(panel);\n\n Polyline polyline = new Polyline();\n polyline.setStart(new Point( 5, 5));\n polyline.addPoint(new Point( 5, 45));\n polyline.addPoint(new Point( 45, 45));\n polyline.addPoint(new Point( 45, 5));\n panel.add(polyline);\n\n Polyline polyline2 = new Polyline();\n polyline2.setStart(new Point( 5, 5));\n polyline2.addPoint(new Point( 45, 45));\n panel.add(polyline2);\n\n Display display = Display.getDefault();\n while( !shell.isDisposed() )\n {\n  if( !display.readAndDispatch() )\n   display.sleep();\n }\n\n}',
 'creation_date': '2002-05-31 09:17 EDT'}
```

**Creation date have to be represented as "year-mont-day hour:minute timezone". If bug report is not duplicate, dup_id have to be None.**

*HINT: lxml parse is faster than html.parser*

In [ ]:
def extract_data_from_page(pagepath):
    """
    Scrap bug report content from bug report html.
    
    :param pagepath: the path of html file.
    :return: 
        {
        "report_id": int,
        "dup_id": int or None (the report id which it is duplicate), 
        "component": string, 
        "product": string, 
        "summary": string, 
        "description": string, 
        "creation_date": string
        }
    """
    pass


## 4.3 - Extract text from HTML


In [ ]:
import os
from multiprocessing import Pool, TimeoutError
from time import time
import json
import tqdm

# Index each report by its id
index_path = os.path.join(FOLDER_PATH, 'bug_reports.json')

if os.path.isfile(index_path):
    # Load threads that webpage content were already extracted.
    report_index = json.load(open(index_path))
else:
    # Extract webpage content

    # This can be slow (around 15 minutes). Test your code with a small sample. lxml parse is faster than html.parser
    files = [os.path.join(PAGE_FOLDER, filename) for filename in os.listdir(PAGE_FOLDER)]
    reports = [extract_data_from_page(f) for f in tqdm.tqdm(files)]
    report_index = dict(((report['report_id'], report) for report in reports ))

    
    json.dump(report_index, open(index_path,'w'))
    

# 5 - Data Preprocessing

Preprocessing is a crucial task in data mining. This task clean and transform the raw data in a format that can better suit data analysis and machine learning techniques. In natural language processing (NLP), *tokenization* and *stemming* are two well known preprocessing steps. Besides these two steps, we will implement an additional step that is designed exclusively for the twitter domain.

## 5.1 - Tokenization

In this preprocessing step, a *tokenizer* is responsible for breaking a text in a sequence of tokens (words, symbols, and punctuations). For instance, the sentence *"It's the student's notebook."* can be split into the following list of tokens: ['It', "'s", 'the', 'student', "'s", 'notebook', '.'].


### 5.1.1 - Question 2 (1 point) 

**Implement** ```tokenize_space_punk``` **that replaces the punctuation to space and then tokenizes the tokens that are separated by whitespace (space, tab, newline). You have to lowercase the tokens.***

In [ ]:
def tokenize_space_punk(text):
    """
    This tokenizer replaces punctuation to spaces and then tokenizes the tokens that are separated by whitespace (space, tab, newline).
    """
    pass
    
        

## 5.2 - Stop words removal

### 5.2.1 - Question 3 (0.5 points)

There are a set of tokens, call stop words, that are not signficant to the similarity comparison since they appear in most of bug report pages. Thus, removing them decreases the vector dimensionality and turns the similarity calculation computationally cheaper. Describe the tokens that can be removed without affecting the similarity comparison. Moreover, implement the function filter_tokens that removes these words from a list of tokens.

**You can use a predefined set of words from a library.**

In [ ]:
def filter_tokens(tokens):
    pass


## 5.3 - Stemming

The process to convert tokens with the same stem (word reduction that keeps word prefixes) to a standard form is called *stemming*. For instance, the word "fishing", "fished" , "fish" and "fisher" are reduced to the stem "fish".


In [ ]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')


word1 = ['I', 'tried', 'different', 'fishes']

print([ stemmer.stem(w) for w in word1])

word2 = ['I', 'will', 'tries', 'only', 'one', 'fishing']
print([ stemmer.stem(w) for w in word2])

### 5.3.1 - Question 4 (0.5 point) 

*Expliquez comment et pourquoi le stemming est utile à notre système.*

# 6 - Data Representation


# 6.1 - Bag of Words (BoW)


Many algorithms only accept inputs that have the same size. However, there are some data types whose sizes are not fixed, for instance, a text can have an unlimited number of words. Imagine that we retrieve two sentences: ”Board games are much better than video games” and ”Monopoly is an awesome game!”. These sentences are respectively named as Sentence 1 and 2. Table below depicts how we could represent both sentences using a fixed representation.

|            | an | are | ! | monopoly | awesome | better | games | than | video | much | board | is | game |
|------------|----|-----|---|----------|---------|--------|-------|------|-------|------|-------|----|------|
| Sentence 1 | 0  | 1   | 0 | 0        | 0       | 1      | 2     | 1    | 1     | 1    | 1     | 0  | 0    |
| Sentence 2 | 1  | 0   | 0 | 1        | 1       | 0      | 0     | 0    | 0     | 0    | 0     | 1  | 1    |

Each column of this table 2.1 represents one of 13 vocabulary words, whereas the rows contains the word
frequencies in each sentence. For instance, the cell in row 1 and column 7 has the value 2
because the word games occurs twice in Sentence 1. Since the rows have always 13 values, we
could use those vectors to represent the Sentences 1 and 2. The table above illustrates a technique called bag-of-words. Bag-of-words represents a document as a vector whose dimensions are equal to the number of times that vocabulary words appeared in the document. Thus, each token will be related to a dimension, i.e., an integer.

*For simplicity, we consider token and word as interchangeable.*



## 6.2 - TF-IDF


Using raw frequency in the bag-of-words can be problematic. The word frequency distribution
is skewed - only a few words have high frequencies in a document ([Zipf law](https://en.wikipedia.org/wiki/Zipf%27s_law)). Consequently, the
weight of these words will be much bigger than the other ones which can give them more
impact on some tasks, like similarity comparison. Besides, words (including
those with high frequency) that appear in most of the documents cannot help discriminating the documents. For instance, the word *of* appears in a significant
portion of reports. Thus, having the word *of* does not make
reports more or less similar. However, the word *terrible* is rarer and reports that
have this word are more likely to be negative. TF-IDF is a technique
to overcome the word frequency disadvantages.


TF-IDF weights the vector using inverse document frequency (IDF) and word frequency, called term frequency (TF).
TF is the local information about how important is a word to a specific document.  IDF measures the discrimination level of the words in a dataset.  Common words in a domain are not helpful to discriminate documents since most of them contain these terms. So, to reduce their relevance in the documents, these words should have low weights in the vectors . 
The following equation calculates the word IDF:
\begin{equation}
	\operatorname{IDF}(t) = \ln\left( \frac{N+1}{\operatorname{df}(t)+1} \right) + 1,
\end{equation}
where $t$ is a token, $N$ is the number of documents in the dataset, and $\operatorname{df}(\cdot)$ is the number of documents that contain a word $i$.
The new weight of a word $t$ in a text is defined as:
\begin{equation}
	w(t) = \operatorname{tf}(t) \times \operatorname{IDF}(t),
\end{equation}
where $\operatorname{tf}(\cdot)$ is the term frequency of word $i$ , i.e, number of times that a word appears in the document. *We call TF-IDF representation when the weights of the BoW represention are computed by means of TF-IDF.*

### 6.2.1 - Question 5 (3.5 points)


*In this TP, we represent each text as list of tuples in which the tuple are composed of the token and its weight computed by TF-IDF. To improve efficiency, we sort the tuples by the tokens. Thus, for instance, Sentence 1 of previous table is represented as: ```[ ("are", 1.4054), ("better", 1.4054), ("board", 1.4054), ("games", 2.8108), ("much", 1.4054), ("than", 1.4054), ("video", 1.4054)]```.*



**Now, you have to implement TF-IDF. Method** ```fit``` **computes IDF based on the textual data in X and method** ```transform``` **transforms a text to a TF-IDF representation.

*Notes*:

- Be careful that ```transform```can receive tokens that were not in X. In this case, consider that $\operatorname{df}(\cdot)=0$.
- For this question, except numpy, you cannot use any external python library (e.g., scikit-learn).

In [ ]:
class TFIDF:
    
    
    def fit(self, X):
        """
        Learn IDF values based on the textual data in X
        
        X:  list of lists of tokens. For instance, X=[['video','awesome', 'The'], ['house', 'The']] 

        :return: None
        """
        pass
            
              

    def transform(self, tokens):
        """
        Transform a list of tokens to TF-IDF representation.
        
        tokens: list of tokens. For instance, tokens=['video','awesome', 'The', 'The'] 
        
        :return: list of tuples. Return the TF-IDF representation of the texts.  
                For instance, [('video', 1.4054), ('awesome', 1.4054), ('The', 2.0)]
        """
        pass
    

## 6.3 - Word embedding

Recently, a new type of representation, called word embeddings, word vectors, or distributed word representation, has been shown very useful for NLP. In word embeddings, words are represented as real, low dimensional and dense vectors. Those vectors describe word positions in a new feature space that retain syntactic and semantic information. In contrast to other representations, word embeddings mitigate the curse of dimensionality and improve
the model capability to handle unknown and rare words in the training. Furthermore,
using distributed word representations, it is possible to perform arithmetical operations and
calculate the distance between words. 
 

### 6.3.1 - Question 6 (3 points)

In this TP, we will use word embeddings to generate a dense representation of text, called text embedding.
In this context, text could contain a sentence or multiple paragraphs.
The text embedding is computed as average of the vectors of the words:
\begin{equation}
	e_s = \frac{1}{|s|} \sum_{t \in s} w_t,
\end{equation}
where $|s|$ is the length of text $s$, $w_t$ is the word embedding of token $t$ in $s$, and $e_s$ is the embedding of $s$.

You will use the pre-trained word embeddings from *glove.42B.300d_clear.txt* in the folder *dataset*.
In each line of this text file, there are the tokens and theirs' vector values. Values and tokens are separated by spaces. In this file, the word embedding dimension is 300.

****Now, you have to implement the method** ```generate_embedding``` **that generates the text embedding. The attribute token2vec is a dictionary that links a word to its vector.****

*Notes:*
- Disconsider words that does not exist in glove.42B.300d_clear.txt
- For this question, except for numpy, you cannot use any external python library (e.g., scikit-learn).

In [ ]:
class TextEmbedding:
    
    def __init__(self):
        word_vec_file = open(word_vec_path)
        self.token2vec = {}
        
        #TODO: load the file content
        pass
    

    def generate_embedding(self, tokens):
        """
        Generate the text embedding as the average of the word embedding in tokens.
        
        Disconsider words that does not exist in glove.42B.300d_clear.txt.

        tokens: list of tokens. E.g., ["are", "better", "board", "much"]

        :return:  text embedding (numpy or list of real numbers)
        """
        pass

# 7 - Pipeline

The pipeline is a sequence of preprocessing steps that transform the raw data to a format that is suitable for your problem. 



## 7.1 - Question 7 (2 points) 

**Now, you have to implement the pipeline following the instructions in the cells below.**


In [ ]:
"""
After the pipeline execution, each report in report_index should contain four keys:
    - summary_tfidf: TF-IDF representation of the summary
    - desc_tfidf: TF-IDF representation of the descripton
    - summary_vec: the text embedding of the summary
    - desc_vec: the text embedding of the description

For example, report 18431 in report_index (report_index[18431]) is:

{'report_id': 18431,
 'dup_id': 27227,
 'component': 'GEF-Legacy Draw2d',
 'product': 'GEF',
 'summary': 'Polylines ignored by FLowLayout',
 'description': 'I tried a poor sample but it\'s not working the way I thought it would be. Look \nat the following source. I have a figure with FlowLayout and add 2 polylines to \nit but the polylines aren\'t arranged in FlowLayout. Both polylines are pushed \ninto the first flow element. Any ideas why?\n\n\n\npublic static void main(String args[])\n{\n Shell shell = new Shell();\n shell.open();\n shell.setText("draw2d Figures");\n\n LightweightSystem lws = new LightweightSystem(shell);\n\n IFigure panel = new Figure();\n panel.setLayoutManager( new FlowLayout(FlowLayout.HORIZONTAL) );\n\n lws.setContents(panel);\n\n Polyline polyline = new Polyline();\n polyline.setStart(new Point( 5, 5));\n polyline.addPoint(new Point( 5, 45));\n polyline.addPoint(new Point( 45, 45));\n polyline.addPoint(new Point( 45, 5));\n panel.add(polyline);\n\n Polyline polyline2 = new Polyline();\n polyline2.setStart(new Point( 5, 5));\n polyline2.addPoint(new Point( 45, 45));\n panel.add(polyline2);\n\n Display display = Display.getDefault();\n while( !shell.isDisposed() )\n {\n  if( !display.readAndDispatch() )\n   display.sleep();\n }\n\n}',
 'creation_date': '2002-05-31 09:17 EDT',
 'summary_vec': array([x.xxxxx, x.xxxxx, ... ]),
 'desc_vec':  array([x.xxxxx, x.xxxxx, ... ]),
 'summary_tfidf': [['flowlayout', x], ['ignor', x], ['polylin', x]],
 'desc_tfidf': [['2', x], ['45', x], ['5', x], ['add', x], ['addpoint', x], ... ]}

"""

"""
Step 1. Tokenize and remove the stop words in the summary and description of each report.
"""
#TODO: implement step 1
pass

In [ ]:

"""
Step 2. Generate the embedding of the summary and description for each report. 
Use the texts preprocessed by step 1 to generate the embeddings.
"""

#TODO: implement step 2
pass

In [ ]:
"""
Step 3. Apply stemming to the tokens generated from Step 1. 
"""
#TODO: implement step 3
pass

In [ ]:
"""
Step 4. Learn the IDF using the summary and description of all the repors in the training set.
You should concatenate the content of these two fields and a report will be considered a document in order to computer the document frequency.
The input of this step should be output of Step 3 (stemmed tokens).
"""
# training_reports_set contains all reports in the training set.
training_reports_set
    
# TODO: implement step 4
pass

In [ ]:
"""
Step 5. Generate the TF-IDF representation of the summary and description for each report.
The input of this step should be output of Step 3 (stemmed tokens).
"""
# TODO: implement step 5
pass

# 8 - Cosine Similarity

In NLP, the cosine similarity is a popular similarity function used to compare the vectors
of documents. This function measures how different is the direction of two vectors, and its
values are between -1 and 1. 

Cosine similarity is defined as:
\begin{equation}
    \operatorname{cos(v, v')} = \frac{\sum_{i=1}^{n} v_i  v_i'}{\sqrt{\sum_{i=1}^{n} v_i} \sqrt{\sum_{i=1}^{n}v_i'}},
\end{equation}
where $v$ and $v'$ are vectors with $n$ dimensions.

## 8.1 - Question 8 (2 points)

**You have to implement two functions:**

 1. ```cosine_sim_tf_idf```: computes the cosine similarity of two representations generated using TF-IDF.
 2. ```cosine_sim_embedding```: computes the cosine similarity of two field embeddings.


*For this question, except for numpy, you cannot use any external python library (e.g., scikit-learn).*



In [ ]:
def cosine_sim_tf_idf(r1, r2):
    """
    r1: TF-IDF representation. 
    r2: TF-IDF representation.

    :return the cosine similarity of r1 and r2
    """
    pass
    



In [ ]:
def cosine_sim_embedding(v1, v2):
    """
    v1: text embedding
    v2: text embedding
   
    :return the cosine similarity of vec1 and vec2
    """
    pass

# 9 - Feature Extraction

We train a logistic regression model to predict whether a pair of reports is duplicate or not. The features employed for the classifion are listed below:

1. Cosine similarity of the TF-IDF representation of summary.
2. Cosine similarity of the TF-IDF representation of description.
2. Cosine similarity of the summary embeddings.
2. Cosine similarity of the description embeddings.
2. A binary feature that is 1.0 when the report are from the same components. Othewise, it is 0.0.
2. A binary feature that is 1.0 when the report are from the same products. Othewise, it is 0.0.


## 9.1 - Question 9 (1 point)
 

**Now, implement** ```extract_features```**that extracts the above features from a pair of reports.** 

*rm_ftr_idxs* is a parameter that allows removing one or more of the original features. If *rm_ftr_idxs=[]*, all features will be used. Otherwise, *rm_ftr_idxs* contains the positions of the features in the previous list to be removed.



In [ ]:
def extract_features(r1, r2, rm_ftr_idxs=[]):
    """
    Extract features from a pair (r1, r2).

    TF-IDF representation of the summary and descriptions can be acessed on r1 and r2 using the keys 'summary_tfidf' and 'desc_tfidf'.
    Summary and description embedding can be acessed on r1 and r2 using the keys 'summary_vec' and 'desc_vec'.


    r1: Dictionary that contains all the information about a report
    r2: Dictionary that contains all the information about a report
    rm_ftr_idxs: Remove features.
        For instance, ftr_opt=[1,4] removes cosine similarity of the TF-IDF representation of summary and the cosine similarity of the description embeddings.

    :return a vector of real numbers. 
    """
    pass

# 10 - Training

In [ ]:
from sklearn.linear_model import LogisticRegression
import numpy as np

#Load labels from the training set
y_train = np.asarray([ y for _, _, y in  training_pairs ])

def train_clf(rm_ftr_idxs=[]):
    # Extract Features
    X_train = np.asarray([ extract_features(report_index[r1], report_index[r2], rm_ftr_idxs) for r1, r2, _ in  training_pairs ])
    return LogisticRegression(random_state=0).fit(X_train, y_train)

# 11 - Question 10  (2.5 points)

Now, it is time to evaluate the classifier. Besides that, you will have to perform an ablation study to measure the effectiveness of each feature.  Ablation study consist in removing a single component from the original architecture, and measuring how much this isolated modification impacts the model performance. The more a component affects the performance, the more is considered to be effective.

For this question, you have to report the classifier accuracy (function ```compute_acc```) for the following configurations:

1. Classifier with all features
2. Classifier without *cosine similarity of the TF-IDF representation of summary and description*.
4. Classifier without *cosine similarity of the summary and description embeddings*.
5. Classifier without *component comparison*.
5. Classifier without *product comparison*.
    
*Also, describe the results and findings.*


**The mark will be penalized by 1.5 point when the accuracy of the classifier with full features (6 features) is lower than 0.92.** 


In [ ]:
import numpy as np


def compute_acc(classifier, X):
    # Compute accuracy
    y = np.asarray([ y for _, _, y in  validation_pairs ])
    return classifier.score(X, y)

    
    
